# Public API V2 <a class="anchor" id="notebook-head"></a>

## Введение <a class="anchor" id="introduction"></a>

Здравствуйте и добро пожаловать в ноутбук, посвященный функциональности 
[Public API V2](https://docs.sbercloud.ru/aicloud/mlspace/concepts/api.html) 
в рамках платформы 
[ML Space](https://sbercloud.ru/ru/aicloud/mlspace).

ML Space предоставляет [REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) 
для того, чтобы пользователи могли частично или полностью автоматизировать пайплайны.

Автоматизация позволяет оптимизировать процесс разработки, избавившись от повторяющихся манипуляций с данными/моделями/etc. с помощью 
[ML Space UI](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/quickstart.html).

Например, у пользователя есть уже заготовленный скрипт, который при незначительных модификациях будет взаимодействовать с ML Space посредством REST API.

В данном ноутбуке описаны примеры использования REST API 

В ноутбуке представлен следующий список операций, которые можно выполнять при помощи REST API:
- Авторизация пользователя 
- Работа с данными
    - Выгрузка содержание бакета
    - Получение credentials бакета
    - Копирование данных из S3 на NFS
    - Копирование данных из NFS на S3
    - Создание и активация S3 коннектора
    - Получение id коннектора
    - Создание переноса с помощью коннектора
    - Получение логов переноса
    - Удаление данных с NFS
- Базовые Docker-образы
    - Получение списка базовых Docker-образов
- Операции с Jupyter Server
    - Создание Jupyter Server
    - Получение списка Jupyter Server 
    - Приостановить Jupyter Server
    - Удалить Jupyter Server 
- Задачи на кластере
    - Получение списка задач
    - Запуск задачи
    - Получение статуса задачи
    - Выгрузка логов задачи
    - Удаление задачи
    - Получение инстанс типов
    - Получение доступных spark конфигураций
- Inference-методы
    - Получение списка сервисов
    - Получение информации по сервису
    - Создание образа для инференс-сервиса
    - Просмотр статуса сборки образа для инференс-сервиса
    - Просмотр логов сборки образа для инференс-сервиса
    - Создание деплоя на основе собранного образа
    - Отправление запроса к созданному сервису

Прежде чем приступить к рассмотрению примеров, необходимо проверить, соблюдены ли предварительные требования:
* <span style="color:green;font-weight:bold">(РЕКОМЕНДУЕТСЯ)</span> Запуск данного ноутбука предпочтительно осуществлять в Jupyter Lab. Это связано с тем, что Jupyter Lab (в отличие от Jupyter Notebook) умеет красиво визуализировать JSON-структуры;

* Наличие рядом с самим ноутбуком JSON-файла с необходимыми авторизационными данными от вашего аккаунта (<span style="color:red;font-weight:bold">ВНИМАНИЕ:</span> это необходимо <span style="color:red;font-weight:bold">исключительно</span> для удобства. Вы можете не создавать такой файл, главное - иметь необходимые данные для авторизации);

* Наличие ```X-Api-Key``` (более подробно описан [здесь](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__profile/profile__develop-func.html#id1));

* Наличие ```X-Workspace-Id``` (используется практически во всех REST-запросах в Public API V2).



# Содержание <a class="anchor" id="toc"></a>

* [Public API V2](#notebook-head)
    * [Введение](#introduction)
* [Все необходимые библиотеки](#all-imports-necessary)
* [Вспомогательные функции и переменные](#auxiliary-stuff)
* [Предварительные требования](#prerequisites)
    * [JSON-файл с необходимыми авторизационными данными](#prerequisites-part-1)
    * [X-Api-Key](#prerequisites-part-2)
    * [X-Workspace-Id](#prerequisites-part-3)
* [Авторизация](#authorization)
* [Работа с данными](#s3)
    * [Выгрузка содержание бакета](#s3-get-bucket-content-structure)
    * [Получение credentials бакета](#s3-credentials)
    * [Копировать данные из S3 на NFS](#s3-copy-to-nfs)
    * [Копировать данные из NFS на S3](#s3-copy-from-s3)
    * [Получение sources и paramaters коннектора](#s3-source-params)
    * [Создание и активация S3 коннектора](#s3-create-try-connector)
    * [Получение id коннектора](#s3-get-id-connector)
    * [Создание переноса с помощью коннектора](#s3-create-transfer)
    * [Получение логов переноса](#s3-transfer-logs)
    * [Удаление данных с NFS](#s3-nfs-delete)
* [Базовые Docker-образы](#docker-images)
    * [Получить список базовых Docker-образов](#docker-images-get)
* [Задачи на кластере](#jobs)
    * [Получение списка задач](#jobs-get)
    * [Запуск задачи](#jobs-run)
    * [Выгрузка логов задачи](#jobs-job-logs)
    * [Удаление задачи](#jobs-delete)
    * [Получение инстанс типов](#jobs-instance-type)
    * [Получение доступных spark конфигураций](#jobs-spark-config)
* [Inference-методы](#inference)
    * [Получить список сервисов](#inference-get-services-list)
    * [Получить информацию по сервису](#inference-get-service-info)
    * [Создать образ для инференс-сервиса](#inference-create-an-image)
    * [Просмотреть статус сборки образа для инференс-сервиса](#inference-image-creation-status)
    * [Просмотреть  логи сборки образа для инференс-сервиса](#inference-image-creation-logs)
    * [Создать деплой на основе собранного образа](#inference-create-service)
    * [Отправить запрос к созданному сервису](#inference-get-a-response-from-service)

# Все необходимые библиотеки ([содержание](#toc))<a class="anchor" id="all-imports-necessary"></a>

Импорт необходимых библиотек

In [21]:
import os

import json

import copy

import requests

from IPython.display import JSON

# Вспомогательные функции и переменные ([содержание](#toc))<a class="anchor" id="auxiliary-stuff"></a>

In [12]:
# def wipe_out_sensitive_information(func):
    
#     def recursive_information_deletion(credentials_dict):
#         sensitive_info_fields = [
#             "password", "id", "x-api-key",
#             "access_key_id", "secret_access_key",
#             "endpoint_url", "bucket_name",
#             "access_token", "refresh_token",
#             "s3_endpoint", "aws_access_key_id",
#             "aws_secret_access_key"
#         ]
        
#         if isinstance(credentials_dict, dict):
#             for k, v in credentials_dict.items():
#                 if isinstance(v, dict):
#                     new_v = recursive_information_deletion(v)
                    
#                     credentials_dict[k] = new_v
#                 elif isinstance(v, str):
#                     if k in sensitive_info_fields:
#                         credentials_dict[k] = "[SENSITIVE INFORMATION, WIPED OUT]"
#                 elif isinstance(v, list):
#                     for i, item in enumerate(v):
#                         new_v_i = recursive_information_deletion(item)
                        
#                         v[i] = new_v_i
                        
#         return credentials_dict

#     def wrapper(credentials_dict):
#         credentials_dict_copy = copy.deepcopy(credentials_dict)
        
#         credentials_dict_copy["[THIS IS A COPY]"] = True
        
#         credentials_dict_copy = recursive_information_deletion(credentials_dict_copy)
        
#         return_value = func(credentials_dict_copy)
        
#         return return_value
    
#     return wrapper

In [13]:
# JSON = wipe_out_sensitive_information(JSON)

In [14]:
host = "https://api.aicloud.sbercloud.ru"

rest_call_template = "{}/public/v2/{}"

# Предварительные требования ([содержание](#toc))<a class="anchor" id="prerequisites"></a>

## JSON-файл с необходимыми авторизационными данными ([содержание](#toc))<a class="anchor" id="prerequisites-part-1"></a>

#### Параметры Client_id и client_secret - long API Keys.
см. [Параметры разработчика](https://sbercloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__profile/profile__develop-func.html)

Если требуется автоматизация использования переменных заполните содержимое файла **credentials_dict.json**.
Файл **credentials_dict.json** при выполнении ячеек должен находиться в том же каталоге, что и сам Jupyter Notebook.

In [15]:
with open("credentials_dict.json", "r") as f:
    credentials_dict = json.load(f)

In [16]:

credentials = {
    "client_id": credentials_dict["client_id"],
    "client_secret": credentials_dict["client_secret"]
}


## X-Api-Key ([содержание](#toc))<a class="anchor" id="prerequisites-part-2"></a>

```X-Api-Key``` (он же ```GWAPI_KEY```) - это клиентский ключ доступа к API (подробнее по [ссылке](https://sbercloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__profile/profile__develop-func.html)).

Он индивидуален для каждого [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html) в рамках аккаунта пользователя.

При создании нового Jupyter Server-а в окружении создается переменная ```GWAPI_KEY```, туда записывается значение ```X-Api-Key```.

Способы получения X-Api-Key:


```python
import os
print(os.environ["GWAPI_KEY"])
```
```bash
echo $GWAPI_KEY
```
```bash
%%bash

echo $GWAPI_KEY
```
```bash
! echo $GWAPI_KEY
```


## X-Workspace-Id ([содержание](#toc))<a class="anchor" id="prerequisites-part-3"></a>

```X-Workspace-Id``` - это уникальный идентификатор вашего [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html), в котором вы работаете.

Более подробно вы можете почитать об этом поле в соответствующем разделе [документации](https://docs.sbercloud.ru/aicloud/mlspace/concepts/deployments__send-http-requests-to-service.html?highlight=workspace%20id#http).

# Авторизация ([содержание](#toc))<a class="anchor" id="authorization"></a>

In [17]:
endpoint = "service_auth"

json_body = json.dumps(credentials)

headers = {
    "X-Api-Key": credentials_dict["workspaces"][0]["x-api-key"],
    "Content-Type": "application/json"
}



In [18]:
url = rest_call_template.format(host, endpoint)
url
response = requests.request(
    "POST",
    url,
    headers=headers,
    data=json_body
)

auth_response_json = json.loads(response.text)

In [19]:
JSON(auth_response_json)

<IPython.core.display.JSON object>

# Работа с данными ([содержание](#toc))<a class="anchor" id="s3"></a>

## Получение credentials бакета ([содержание](#toc))<a class="anchor" id="s3-credentials"></a>

Подробнее в [документации](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/s3/operation/get_s3_credentials_public_v2_s3_credentials_get)

In [ ]:

endpoint = "s3/credentials"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url



In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers,
    
)
job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

In [ ]:
endpoint = 's3/credentials'

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}
json_body = {
  "s3_namespace": "null",
  "s3_endpoint": credentials_dict["workspaces"][0]["endpoint_url"],
  "access_key_id": credentials_dict["workspaces"][0]["access_key_id"],
  "security_key": credentials_dict["workspaces"][0]["secret_access_key"]
}


In [ ]:
url = rest_call_template.format(host, endpoint)
url

response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
JSON(job_response_json)

## Копирование данных из S3 на NFS (<span style="color:red;font-weight:bold">Deprecated</span>)  ([содержание](#toc))<a class="anchor" id="s3-copy-to-nfs"></a>  

In [ ]:

endpoint = "data_transfer/v1/transfers/s3_to_nfs"
headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "source_path": "",
    "destination_path": "",
    "strategy": "write_all",
    "to_region": "CCE",
    "s3_credentials": {
      "endpoint": credentials_dict["workspaces"][0]["endpoint_url"],
      "bucket":credentials_dict["workspaces"][0]["bucket_name"],
      "access_key_id": credentials_dict["workspaces"][0]["access_key_id"],
      "security_key": credentials_dict["workspaces"][0]["secret_access_key"]
  }
}


url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Копирование данных из NFS на S3 (<span style="color:red;font-weight:bold">Deprecated</span>) ([содержание](#toc))<a class="anchor" id="s3-copy-from-s3"></a>

In [ ]:
endpoint = "data_transfer/v1/transfers/nfs_to_s3"
headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "source_path": "model.pkl",
    "destination_path": "",
    "strategy": "write_all",
    "from_region": "CCE",
    "s3_credentials": {
      "endpoint": credentials_dict["workspaces"][0]["endpoint_url"],
      "bucket":credentials_dict["workspaces"][0]["bucket_name"],
      "access_key_id": credentials_dict["workspaces"][0]["access_key_id"],
      "security_key": credentials_dict["workspaces"][0]["secret_access_key"]
  }
}


url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Получение source и parameters коннектора ([содержание](#toc))<a class="anchor" id="s3-source-params"></a>

In [ ]:
endpoint = "data_transfer/v2/connectors/sources"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url


In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Создание и активация коннектора ([содержание](#toc))<a class="anchor" id="s3-create-try-connector"></a>

 [Создание](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/data-transfer/operation/new_connector_public_v2_data_transfer_v2_connectors_post) и [активация](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/data-transfer/operation/try_connector_public_v2_data_transfer_v2_connectors__id___try_get) коннектора

In [ ]:
endpoint = "data_transfer/v2/connectors"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "name": "",
    "source_type": "s3custom",
    "favorite" : False,
    "parameters": {
        "endpoint":
        "bucket":
        "access_key_id"
        "security_key"
    }
    }


url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

In [ ]:
endpoint = "data_transfer/v2/connectors/{id_}/try"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

jobs_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Получение id коннектора ([содержание](#toc))<a class="anchor" id="#s3-get-id-connector"></a>

Подробнее в [документации](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/data-transfer/operation/connector_parameters_public_v2_data_transfer_v2_connectors__id___get)

In [ ]:
endpoint = "data_transfer/v2/connectors"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url




In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Создание переноса с помощью коннектора ([содержание](#toc))<a class="anchor" id="s3-create-transfer"></a> 

Подробнее в [документации](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/data-transfer/operation/create_transfer_public_v2_data_transfer_v2_transfer_post)

In [ ]:
endpoint = "data_transfer/v2/transfer"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "cluster_name": "pd11",
    "connector_id": ,
    "destination_connector_id": "",
    "name":"",
    "strategy": "",
    "favorite": False,
    "query": ""
    }


url = rest_call_template.format(host, endpoint)
url


In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Получение логов переноса ([содержание](#toc)) <a class="anchor" id="s3-transfer-logs"></a>

Подробнее в [документации](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/data-transfer/operation/transfer_logs_public_v2_data_transfer_v2_transfer_aicloud_logs__transfer_id__get)

In [ ]:
endpoint = "data_transfer/v2/transfer/aicloud-logs/{transfer_id}"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url



In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

jobs_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Удаление данных с NFS ([содержание](#toc)) <a class="anchor" id="s3-nfs-delete"></a>

Подробнее в [документации](https://api.aicloud.sbercloud.ru/public/v2/redoc#tag/storage/operation/delete_storage_entity_public_v2_service_storage_erase_delete)

In [ ]:
endpoint = "service/storage/erase"

headers = {
    "Content-Type": "application/json",
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "path": "/home/jovyan/model.pkl",
    "verbose": False,
    "region": "A100-MT"
    }


url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "DELETE",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

# Базовые Docker-образы ([содержание](#toc))<a class="anchor" id="docker-images"></a>

## Получение списка базовых Docker-образов ([содержание](#toc))<a class="anchor" id="docker-images-get"></a>

In [ ]:
endpoint = "service/base_mt_images"

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url
)

response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

In [ ]:
#Получить список инстанс типов
endpoint = "configs?cluster_type=MT"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "cluster_type":"MT"
}  


url = rest_call_template.format(host, endpoint)
url


In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

# Операции с Jupyter Server ([содержание](#toc))<a class="anchor" id="s3-get-bucket-content-structure"></a>

## Создание jupyter notebook ([содержание](#toc))<a class="anchor" id="s3-get-bucket-content-structure"></a>

In [ ]:


endpoint = "notebooks/v1/{namespace}/notebook"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "image": {
        "name":"cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0",
        "tag":"0.0.32.3",
        "type":"default"
    },
    "name": "apijs",
    "region": "CCE-MT",
    "instance_type":"m3.large.8.linux"
    }


url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

# Задачи на кластере ([содержание](#toc))<a class="anchor" id="jobs"></a>

## Получение списка задач ([содержание](#toc))<a class="anchor" id="jobs-get"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

jobs_response_json = json.loads(response.text)

In [ ]:
JSON(jobs_response_json)

## Запуск задачи ([содержание](#toc))<a class="anchor" id="jobs-run"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "base_image": "cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0:0.0.32.3",
    "script": "/home/jovyan/test.py",
    "n_gpus": 1,
    "n_workers":1,
    "warm_cache": False,
    "region":"SR002-MT",
    "instance_type":"a100.1gpu.80vG.12C.96G"
    }

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Получение статуса задачи ([содержание](#toc))<a class="anchor" id="s3-get-bucket-content-structure"></a>

### Для задач обучения существуют следующие статусы:

Completed — успешно завершено для обучения на GPU.

Deleted — задача удалена.

Failed — ошибка в процессе запуска, обучения или в скрипте.

Pending — задача обучения находится в очереди на запуск.

Running — задача обучения запущена.

Stopped — остановлена пользователем.

Succeeded — успешно завершено для обучения на CPU.

Terminated — задача остановлена.

Подробнее в [документации](https://sbercloud.ru/ru/docs/aicloud/mlspace/concepts/client-lib__job.html#client-lib-job-status)

In [ ]:

endpoint = "jobs/{}".format(job_response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

## Выгрузка логи задачи ([содержание](#toc))<a class="anchor" id="jobs-job-logs"></a>

Доступно после перехода задачи в статус "Выполняется(Running)"

In [ ]:
endpoint = "jobs/{}/logs".format(job_response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

In [ ]:
JSON(job_response_json)

## Удаление задачи ([содержание](#toc))<a class="anchor" id="jobs-delete"></a>

In [ ]:
endpoint = "jobs/{}".format(job_response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "DELETE",
    url,
    headers=headers
)

print(response.text)

## Получение инстанс типов ([содержание](#toc))<a class="anchor" id="jobs-instance-type"></a>

Подробнее в [документации](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/api.html#instances-types)

In [41]:
endpoint = "configs?cluster_type=MT"     

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

'https://api.aicloud.sbercloud.ru/public/v2/configs?cluster_type=MT'

In [47]:
response = requests.request(
    "GET",
    url,
    headers=headers
)
job_response_json = json.loads(response.text)

In [48]:
JSON(job_response_json)

<IPython.core.display.JSON object>

## Получение доступных spark конфигураций ([содержание](#toc))<a class="anchor" id="jobs-spark-config"></a>

Подробнее в [документации](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/api.html#instances-types) 

In [49]:
endpoint = "configs/spark"     

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

'https://api.aicloud.sbercloud.ru/public/v2/configs/spark'

In [50]:
response = requests.request(
    "GET",
    url,
    headers=headers
)
job_response_json = json.loads(response.text)

In [51]:
JSON(job_response_json)

<IPython.core.display.JSON object>

# Inference-методы ([содержание](#toc))<a class="anchor" id="inference"></a>

## Получить список сервисов ([содержание](#toc))<a class="anchor" id="inference-get-services-list"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "services": response_json
}

In [ ]:
JSON(response_json)

## Получить информацию по сервису ([содержание](#toc))<a class="anchor" id="inference-get-service-info"></a>

In [ ]:
endpoint = "inference/v1/{}".format(response_json["services"][2]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "service": response_json
}

In [ ]:
JSON(response_json)

## Создать образ для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-create-an-image"></a>

In [ ]:
s3.upload_file(
    "/home/jovyan/requirements.txt",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/requirements.txt",
)

In [ ]:
s3.upload_file(
    "/home/jovyan/dummy_mirror_serving_script.py",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/dummy_mirror_serving_script.py",
)

In [ ]:
endpoint = "inference/build/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "base_image": "registry.aicloud.sbcp.ru/base/horovod-cuda10.0-tf1.15.0",
    "run_script": "dummy_mirror_serving_script.py",
    "requirements_path": "requirements.txt",
    "artifacts_directory": "{}/inference_folder/".format(credentials_dict["workspaces"][0]["bucket_name"]),
    "AWS_ACCESS_KEY_ID": credentials_dict["workspaces"][0]["access_key_id"],
    "AWS_SECRET_ACCESS_KEY": credentials_dict["workspaces"][0]["secret_access_key"],
    "S3_ENDPOINT": credentials_dict["workspaces"][0]["endpoint_url"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

## Просмотреть статус сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-status"></a>

In [ ]:
endpoint = "service/jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
JSON(json.loads(response.text))

## Просмотреть логи сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-logs"></a>

In [ ]:
endpoint = "service/jobs/{}/logs".format(response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

## Создать деплой на основе собранного образа ([содержание](#toc))<a class="anchor" id="inference-create-service"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "image": response_json["image"],
    "resources": {
        "cpu": 1,
        "memory": 10,
        "gpu": "1"
    },
    "replicas": {
        "min": 1,
        "max": 2
    }
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
print(response_json["metadata"]["name"])

## Отправить запрос к созданному сервису ([содержание](#toc))<a class="anchor" id="inference-get-a-response-from-service"></a>

In [ ]:
endpoint = "inference/v1/predict/{}/{}/".format(response_json["metadata"]["name"], response_json["metadata"]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["x-api-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "instances": [
        {
            "image_link": "1.jpg"
        },
        {
            "image_link": "2.jpg"
        },
        {
            "image_link": "3.jpg"
        }
    ]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
JSON(json.loads(response.text))